In [1]:
pip install llama-index-llms-cohere


Note: you may need to restart the kernel to use updated packages.


In [3]:
import llama_index
from llama_index.llms.cohere import Cohere

import json
import requests
import uuid
from datetime import datetime, timezone

def hello_http(prompt):


    api_key = "RAjqpE9uTVFfTTaF0RIGUM4PaKBeQ7insNIu9KLC"

    llm = Cohere(model="command-r-08-2024", api_key=api_key)

    resp = llm.complete(prompt)

    #print(resp)

    return resp


ModuleNotFoundError: No module named 'deprecated'

In [4]:
def salvar_chat(id_usuario, message, nm_chat):

    ORACLE_API_URL = "https://g56e15c6771c555-nutriaidb.adb.sa-saopaulo-1.oraclecloudapps.com/ords/admin/chats/"
    
    print("Iniciando salvamento do chat no Oracle...")

    data_de_criacao_iso = datetime.now(timezone.utc).isoformat()

    # Payload (corpo da requisição) para a API do Oracle
    oracle_payload = {
        "uid_usuario_chat":"uid-002",
        "nm_chat": nm_chat,
        "ds_mensagens_chat": message,
        "dt_criacao_chat": data_de_criacao_iso
    }

    headers = {"Content-Type": "application/json"}
    
    # Faz a requisição POST e levanta uma exceção em caso de erro (4xx ou 5xx)
    oracle_response = requests.post(ORACLE_API_URL, data=json.dumps(oracle_payload), headers=headers)
    oracle_response.raise_for_status()
    
    print(f"Chat salvo com sucesso. ID: {id_usuario}")
    return id_usuario


In [5]:
resp = hello_http("Olá como pode me ajudar?")

salvar_chat("uid-002", "teste domingão", "João")

print(resp.text)

Iniciando salvamento do chat no Oracle...
Chat salvo com sucesso. ID: uid-002
Olá! Como um assistente de IA, estou aqui para ajudar com uma ampla gama de tarefas e perguntas. Você pode me pedir ajuda com praticamente qualquer coisa, desde informações gerais até orientação em áreas específicas. Aqui estão algumas maneiras pelas quais posso ajudá-lo:

1. Respostas a Perguntas: Você pode me perguntar sobre qualquer tópico, desde ciência e história até cultura popular e hobbies. Posso fornecer informações precisas e atualizadas.

2. Explicações: Se você tiver dúvidas sobre conceitos ou ideias complexas, posso explicar de forma clara e concisa, facilitando a compreensão.

3. Sugestões e Ideias: Precisa de inspiração ou sugestões para um projeto, hobby ou atividade? Posso oferecer ideias criativas e alternativas para ajudá-lo a explorar novas possibilidades.

4. Resumos e Sínteses: Se você tiver muito conteúdo para absorver, posso resumir textos, artigos ou livros para ajudá-lo a entender os

In [4]:
ORACLE_API_URL = "https://g56e15c6771c555-nutriaidb.adb.sa-saopaulo-1.oraclecloudapps.com/ords/admin/chats/"

def get_chat_date(chat_record):
    """
    Função auxiliar segura para extrair e converter a data de um registro de chat.
    Retorna um objeto datetime válido ou uma data mínima se a data for nula ou inválida.
    """
    # Usar .get() é mais seguro pois retorna None se a chave não existir
    date_str = chat_record.get('dt_criacao_chat')

    # Se a data for Nula ou uma string vazia, trata como a data mais antiga possível
    if not date_str:
        return datetime.min

    # Tenta converter a data (primeiro o formato ISO, depois o formato customizado)
    try:
        # Opção A: Formato ISO 8601 (ex: "2025-07-26T18:30:00Z")
        return datetime.fromisoformat(date_str.replace('Z', '+00:00'))
    except (ValueError, TypeError):
        try:
            # Opção B: Formato customizado (ex: "26/07/2025, 03:00:29 PM")
            return datetime.strptime(date_str, "%d/%m/%Y, %I:%M:%S %p")
        except (ValueError, TypeError):
            # Se nenhum formato funcionar, trata como a data mais antiga
            print(f"AVISO: Não foi possível converter a data '{date_str}'. Tratando como antigo.")
            return datetime.min

def buscar_e_filtrar_chats(id_chat=None, uid_usuario=None, nm_chat=None):
    """
    Busca todos os chats da API Oracle, filtra em memória e os ordena por data de forma segura.
    """
    print("Buscando todos os chats da API...")
    try:
        response = requests.get(ORACLE_API_URL)
        response.raise_for_status()

        todos_os_chats = response.json().get('items', [])
        print(f"Encontrados {len(todos_os_chats)} chats no total.")

        # A lógica de filtragem continua a mesma
        resultados_filtrados = []
        for chat in todos_os_chats:
            match_id = (id_chat is None) or (str(chat.get('id_chat')) == str(id_chat))
            match_uid = (uid_usuario is None) or (chat.get('uid_usuario_chat') == uid_usuario)
            match_nm = (nm_chat is None) or (nm_chat.lower() in str(chat.get('nm_chat', '')).lower())
            if match_id and match_uid and match_nm:
                resultados_filtrados.append(chat)
        
        print(f"Após a filtragem, restaram {len(resultados_filtrados)} chats.")

        # --- ORDENAÇÃO SEGURA USANDO A FUNÇÃO AUXILIAR ---
        # A função `get_chat_date` lida com todos os casos de erro (nulos, formatos, etc.)
        resultados_ordenados = sorted(resultados_filtrados, key=get_chat_date)

        print("Chats ordenados com sucesso.")
        return resultados_ordenados

    except requests.exceptions.RequestException as e:
        print(f"Erro ao fazer a requisição para a API: {e}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

In [11]:

import requests

import json
import requests
import uuid
from datetime import datetime, timezone

print("\n--- Exemplo 2: Buscar um chat com um nome específico ---")
chats_por_nome = buscar_e_filtrar_chats(nm_chat="Defict Calórico", uid_usuario="uid-001")
if chats_por_nome:
    print(f"Encontrado: {chats_por_nome}")


--- Exemplo 2: Buscar um chat com um nome específico ---
Buscando todos os chats da API...
Encontrados 14 chats no total.
Após a filtragem, restaram 2 chats.
Chats ordenados com sucesso.
Encontrado: [{'id_chat': 73, 'uid_usuario_chat': 'uid-001', 'nm_chat': 'Defict Calórico', 'ds_mensagens_chat': 'essa é a numero 2', 'dt_criacao_chat': '2025-08-02T17:40:12Z', 'id_paciente': 22, 'links': [{'rel': 'self', 'href': 'https://g56e15c6771c555-nutriaidb.adb.sa-saopaulo-1.oraclecloudapps.com/ords/admin/chats/73'}]}, {'id_chat': 74, 'uid_usuario_chat': 'uid-001', 'nm_chat': 'Defict Calórico', 'ds_mensagens_chat': 'Claro! Estou aqui para ajudar. \n\nQual é a dúvida específica do seu paciente?', 'dt_criacao_chat': '2025-08-02T17:40:14Z', 'id_paciente': 22, 'links': [{'rel': 'self', 'href': 'https://g56e15c6771c555-nutriaidb.adb.sa-saopaulo-1.oraclecloudapps.com/ords/admin/chats/74'}]}]


In [14]:
# --- Passo 1: Extrair o essencial para uma nova lista ---
chats_simplificados = []
for chat in chats_por_nome:
    # Pega a mensagem
    mensagem = chat['ds_mensagens_chat']
    
    # Pega a data (que está como texto)
    data_texto = chat['dt_criacao_chat']
    
    # Converte o texto da data para um objeto datetime do Python
    # O .replace('Z', '+00:00') ajuda o Python a entender o fuso horário UTC
    data_objeto = datetime.fromisoformat(data_texto.replace('Z', '+00:00'))

    tempo_formatado = data_objeto.strftime('%d/%m/%Y %H:%M:%S')
    
    # Adiciona um novo dicionário com os dados limpos à nossa lista
    chats_simplificados.append({
        'mensagem': mensagem,
        'tempo': tempo_formatado
    })

# --- Passo 2: Ordenar a lista pela chave 'tempo' ---
# A função sorted() cria uma nova lista ordenada.
# O 'key=lambda item: item['tempo']' diz para ordenar usando o valor da chave 'tempo'.
chats_ordenados = sorted(chats_simplificados, key=lambda item: item['tempo'])

print(chats_ordenados)


# # --- Passo 3: Exibir o resultado de forma legível ---
# print("--- Histórico do Chat (Do mais antigo para o mais novo) ---")
# for chat in chats_ordenados:
#     # Formata a data para um padrão mais amigável (Dia/Mês/Ano Hora:Minuto:Segundo)
#     tempo_formatado = chat['tempo'].strftime('%d/%m/%Y %H:%M:%S')
    
#     print(f"Data: {tempo_formatado}")
#     print(f"Mensagem: {chat['mensagem']}")
#     print("-" * 20) # Apenas uma linha para separar as mensagens

[{'mensagem': 'essa é a numero 2', 'tempo': '02/08/2025 17:40:12'}, {'mensagem': 'Claro! Estou aqui para ajudar. \n\nQual é a dúvida específica do seu paciente?', 'tempo': '02/08/2025 17:40:14'}]
